# Loading Necessary Packages

In [7]:
import numpy as np
import pandas as pd
import time

import warnings
warnings.filterwarnings('ignore')

# Reading the data

In [8]:
data = pd.read_csv('Film_Permits.csv')

# Extracting only Shooting Permits and Filtering data

We want to only look at shooting permits so we filter the data accordingly. We also get rid of columns that we will not be using in our analysis. In other words, we only keep 'EventID', 'StartDateTime', 'EndDateTime', 'Borough', and 'PolicePrecinct(s)'.

In [9]:
shoot_permits = data.loc[data['EventType'] == 'Shooting Permit'].copy()
filtered_data = shoot_permits[['EventID', 'StartDateTime', 'EndDateTime', 'Borough', 'PolicePrecinct(s)']]
filtered_data.reset_index(drop = True, inplace = True)

# Extracting different parts of the start dates

To help simplify working with date and time data, we make new columns to extract different parts of the date and time data in the dataset and format them for our convenience and human readability.

In [10]:
# Formatting date-time as month/date/year hour:minute:second AM or PM
filtered_data['StartDateTime_c'] = pd.to_datetime(filtered_data['StartDateTime'], format = '%m/%d/%Y %I:%M:%S %p')
filtered_data['EndDateTime_c'] = pd.to_datetime(filtered_data['EndDateTime'], format = '%m/%d/%Y %I:%M:%S %p')

# Extracting parts of the date
filtered_data['StartDate']      = filtered_data['StartDateTime_c'].apply(lambda d: d.date())
filtered_data['StartHour']      = filtered_data['StartDateTime_c'].apply(lambda d: d.hour)
filtered_data['StartMonth']     = filtered_data['StartDateTime_c'].apply(lambda d: d.month)
filtered_data['StartYear']      = filtered_data['StartDateTime_c'].apply(lambda d: d.year)
filtered_data['StartYearMonth'] = filtered_data.apply(lambda d: str(d['StartYear']) + '-' + str(d['StartMonth']).zfill(2), axis = 1)


In [11]:
filtered_data.to_csv('filtered_data.csv', index = False)

# Exploding Police Precincts

The police precincts are comma separated values in cases where shooting spanned multiple precincts. We need to do something in order to explode these lists into multiple rows so we can easily work with them in our visualization. 

In [5]:
precincts_new = [] # where we will store our list of exploded precincts
precincts = filtered_data['PolicePrecinct(s)']
precincts_list = list(precincts)

for p in precincts_list:
    p = str(p)
    pp = p.split(", ")
    precincts_new.append(pp)

precincts_df = pd.DataFrame(precincts_new) # creating dataframe out of precincts list

# Now we create a new dataframe that contains duplicate EventIDs if the shooting takes place in more than one precinct
We have one row for each police precinct number. 

In [6]:
explode = pd.DataFrame(columns = ['EventID'])
for i in range(len(filtered_data)):
    for j in range(0, 11): # we saw that there were at most 11 precincts for a single EventID
        if not(precincts_df[j][i] is None): # automatically gets rid of any missing precinct values
            explode = explode.append({'EventID': filtered_data['EventID'][i],
                                      'PolicePrecinct': precincts_df[j][i]}, ignore_index = True)

# We create a brand new CSV file out of the exploded police precinct data

In [7]:
explode.to_csv('precincts_exploded.csv', index = False)

In [8]:
pp_data = pd.read_csv('precincts_exploded.csv')

In [9]:
final_dataset = pp_data.merge(filtered_data[['EventID', 'Borough', 'StartDate', 'StartMonth', 'StartYear', 'StartYearMonth']], on = 'EventID')

In [10]:
final_dataset.to_csv('final_permits.csv', index = False)

In [4]:
fin_csv = pd.read_csv('final_permits.csv')
manhattan = fin_csv[(fin_csv['Borough'] == 'Manhattan')]

In [8]:
years = manhattan['StartYear'].unique()

year_precinct = []
for y in years:
    for pp in [1.0, 5.0, 6.0, 7.0, 9.0, 10.0, 13.0, 14.0, 17.0, 18.0, 19.0, 20.0, 22.0, 23.0, 24.0, 25.0, 26.0, 28.0, 30.0, 32.0, 33.0, 34.0]:
        count = len(manhattan[(manhattan['StartYear'] == int(y)) & (manhattan['PolicePrecinct'] == pp)])
        year_precinct.append([y, pp, count])

In [9]:
year_precinct_count = pd.DataFrame(year_precinct, columns = ['Year', 'Precinct', 'NumberofPermits']) 

# The CSV file we will use for the heatmap

In [11]:
year_precinct_count.to_csv('manhattan_number_of_permits.csv', index = False)